In [1]:
import matplotlib.pyplot as plt 
from matplotlib import font_manager as fm 
from matplotlib.pyplot import gca
import matplotlib as mpl
from cycler import cycler
import math

linestyle_tuple = [
     ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or '.'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),
     
     ('loosely dotted',        (0, (1, 10))),
     ('dotted',                (0, (1, 1))),
     ('densely dotted',        (0, (1, 1))),

     ('loosely dashed',        (0, (5, 10))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),

     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),

     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))
     
     ]

linestyle_dict = {k : v for k,v in linestyle_tuple}

def setup_global():
    font_entry = fm.FontEntry(
        fname = './gillsans.ttf',
        name='gill-sans')

    # set font
    fm.fontManager.ttflist.insert(0, font_entry) 
    mpl.rcParams['font.family'] = font_entry.name 

    mpl.use('Agg')
    mpl.rcParams['axes.spines.right'] = False
    mpl.rcParams['axes.spines.top'] = False
    mpl.rcParams['font.size'] = 16
    mpl.rcParams['lines.linewidth'] = 2
    mpl.rcParams['lines.markersize'] = 5
    #mpl.rcParams['hatch.linewidth'] = 0.05

# def setup_local():
#     plt.clf()
#     gca().yaxis.grid(linestyle='dotted', which="both")
#     gca().spines['left'].set_color('#606060')
#     gca().spines['bottom'].set_color('#606060')
#     global_cycler = cycler(color=get_colors()[:len(get_linestyles())]) + cycler(linestyle=get_linestyles()) 
#     gca().set_prop_cycle(global_cycler)
    
def setup_local(axis=None):
    if axis is None:
        plt.clf()
        axis = gca()
    
    axis.yaxis.grid(linestyle='dotted', which="both")
    axis.spines['left'].set_color('#606060')
    axis.spines['bottom'].set_color('#606060')
    global_cycler = cycler(color=get_colors()[:len(get_linestyles())]) + cycler(linestyle=get_linestyles()) 
    axis.set_prop_cycle(global_cycler)

def set_aspect_ratio(ratio=3/5, logx=None, logy=None, axis=None):
    if axis is None:
        axis = gca() 
    xleft, xright = axis.get_xlim()
    if logx is not None:
        xleft = math.log(xleft, logx)
        xright = math.log(xright, logx)
    ybottom, ytop = axis.get_ylim()
    if logy is not None:
        ytop = math.log(ytop, logy)
        print(ytop, ybottom)
        ybottom = math.log(ybottom, logy)
    axis.set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)

# def set_aspect_ratio(ratio=3/5, logx=None, logy=None): 
#     xleft, xright = gca().get_xlim()
#     if logx is not None:
#         xleft = math.log(xleft, logx)
#         xright = math.log(xright, logx)
#     ybottom, ytop = gca().get_ylim()
#     if logy is not None:
#         ytop = math.log(ytop, logy)
#         print(ytop, ybottom)
#         ybottom = math.log(ybottom, logy)
#     gca().set_aspect(abs((xright-xleft)/(ybottom-ytop))*ratio)

def get_colors():
    return ['#D55E00', 
            '#009E73',
            '#0072B2', 
            '#CC79A7', 
            '#000000', 
            '#E03A3D',
            '#F0E442',]

def get_colors_dict():
    return {
            'orange': '#D55E00', 
            'green': '#009E73',
            'blue': '#0072B2', 
            'purple': '#CC79A7', 
            'black': '#000000', 
            'red': '#E03A3D',
            'yellow': '#F0E442',
            }

def get_hatches():
    return ['xxx', '|||', '\\\\', 'xxxxxx',  '///////', '+']

def get_linestyles():
    return [
            linestyle_dict['solid'], 
            linestyle_dict['dotted'], 
            linestyle_dict['dashed'],
            linestyle_dict['dashdotted'],
            linestyle_dict['dashdotdotted'],
            linestyle_dict['densely dashdotted'],
            
    ]

def get_linestyles_dict():
    return linestyle_dict

def get_markers():
    return ['^', 'd', 's', 'o', 'x']

In [2]:
import os
import re
import sys

def process_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Extract GPU count from the content
    gpu_count_match = re.search(r'Number of GPUs:\s*(\d+)', content)
    if gpu_count_match:
        gpu_count = gpu_count_match.group(1)
    else:
        print(f"Warning: Could not find GPU count in {file_path}")
        return None, None
    
    # Extract times - specifically matches numbers before "seconds"
    times = re.findall(r'(\d+\.?\d*)\s+seconds', content)
    times = [float(time) for time in times]
    
    return gpu_count, times

def process_directory(benchmark_path, data_type):
    results = {}

    for file in os.listdir(benchmark_path):
        if file.endswith('.txt'):
            file_path = os.path.join(benchmark_path, file)
            gpu_count, times = process_file(file_path)
            
            key = f"{data_type}_times{gpu_count}"
            results[key] = times
    
    return results

def print_results(results):
    for key, times in sorted(results.items()):
        formatted_times = [f"{time:.6f}" for time in times]
        print(f"{key} = [{', '.join(formatted_times)}]")

def main(nccl_benchmark_path, mpi_benchmark_path):
    nccl_results = process_directory(nccl_benchmark_path, 'nccl')
    mpi_results = process_directory(mpi_benchmark_path, 'mpi')
    
    all_results = {**nccl_results, **mpi_results}
    print_results(all_results)

print("NCCL & MPI ALL-GATHER")
nccl_benchmark_path, mpi_benchmark_path = "./nccl_perlm/all-gather/benchmarks", "./mpi_perlm/all-gather/benchmarks"
main(nccl_benchmark_path, mpi_benchmark_path)

print("NCCL & MPI ALL-REDUCE")
nccl_benchmark_path, mpi_benchmark_path = "./nccl_perlm/all-reduce/benchmarks", "./mpi_perlm/all-reduce/benchmarks"
main(nccl_benchmark_path, mpi_benchmark_path)

print("NCCL & MPI REDUCE-SCATTER")
nccl_benchmark_path, mpi_benchmark_path = "./nccl_perlm/reduce-scatter/benchmarks", "./mpi_perlm/reduce-scatter/benchmarks"
main(nccl_benchmark_path, mpi_benchmark_path)

NCCL & MPI ALL-GATHER
mpi_times128 = [0.003072, 0.005233, 0.008462, 0.015449, 0.030325, 0.060131, 0.190401]
mpi_times16 = [0.002476, 0.003571, 0.007188, 0.014909, 0.030427, 0.061974, 0.150229]
mpi_times32 = [0.000814, 0.001392, 0.002735, 0.003736, 0.007699, 0.014426, 0.030468, 0.063086, 0.172433]
mpi_times64 = [0.001616, 0.003051, 0.004224, 0.008058, 0.015085, 0.029593, 0.063129, 0.185107]
mpi_times8 = [0.000804, 0.001514, 0.003268, 0.006800, 0.013764, 0.027832, 0.055076, 0.103476]
nccl_times128 = [0.002077, 0.002368, 0.002832, 0.004504, 0.007551, 0.014982, 0.028604, 0.056227]
nccl_times16 = [0.000643, 0.000944, 0.001838, 0.003452, 0.007084, 0.013794, 0.026821, 0.052760]
nccl_times32 = [0.000528, 0.000604, 0.000701, 0.001044, 0.002055, 0.004240, 0.006949, 0.014221, 0.027622]
nccl_times64 = [0.001230, 0.001226, 0.001381, 0.002098, 0.003764, 0.007649, 0.014257, 0.027941]
nccl_times8 = [0.000325, 0.000482, 0.000881, 0.001679, 0.003206, 0.006338, 0.012452, 0.024147]
NCCL & MPI ALL-REDUCE
m

In [3]:
# MPI-NCCL-ALL-GATHER
import numpy as np
import matplotlib.pyplot as plt 
from functools import partial

setup_global()
colors_dict = get_colors_dict()
linestyles = get_linestyles_dict()
markers = get_markers()

setup_local()

gpus = [8, 16, 32, 64, 128]
x1 = [2, 4, 8, 16, 32, 64, 128, 256]
x2 = [0.25, 0.5, 1, 2, 4, 8, 16, 32, 64]

mpi_times128 = [0.003072, 0.005233, 0.008462, 0.015449, 0.030325, 0.060131, 0.190401]
mpi_times16 = [0.002476, 0.003571, 0.007188, 0.014909, 0.030427, 0.061974, 0.150229]
mpi_times32 = [0.000814, 0.001392, 0.002735, 0.003736, 0.007699, 0.014426, 0.030468, 0.063086, 0.172433]
mpi_times64 = [0.001616, 0.003051, 0.004224, 0.008058, 0.015085, 0.029593, 0.063129, 0.185107]
mpi_times8 = [0.000804, 0.001514, 0.003268, 0.006800, 0.013764, 0.027832, 0.055076, 0.103476]
nccl_times128 = [0.002077, 0.002368, 0.002832, 0.004504, 0.007551, 0.014982, 0.028604, 0.056227]
nccl_times16 = [0.000643, 0.000944, 0.001838, 0.003452, 0.007084, 0.013794, 0.026821, 0.052760]
nccl_times32 = [0.000528, 0.000604, 0.000701, 0.001044, 0.002055, 0.004240, 0.006949, 0.014221, 0.027622]
nccl_times64 = [0.001230, 0.001226, 0.001381, 0.002098, 0.003764, 0.007649, 0.014257, 0.027941]
nccl_times8 = [0.000325, 0.000482, 0.000881, 0.001679, 0.003206, 0.006338, 0.012452, 0.024147]


i=0
plt.plot(x1,nccl_times8, label="8 GPUs NCCL", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1,mpi_times8, label="8 GPUs MPI", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["dashed"])

i=1
plt.plot(x1, nccl_times16, label="16 GPUs NCCL", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[:-1], mpi_times16, label="16 GPUs MPI", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["dashed"])

i=2
plt.plot(x2, nccl_times32, label="32 GPUs NCCL", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x2, mpi_times32, label="32 GPUs MPI", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["dashed"])

i=3
plt.plot(x2[:-1], nccl_times64, label="64 GPUs NCCL", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x2[:-1], mpi_times64, label="64 GPUs MPI", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["dashed"])

i=4
plt.plot(x2[:-1], nccl_times128, label="128 GPUs NCCL", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x2[:-2], mpi_times128, label="128 GPUs MPI", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["dashed"])

plt.xscale('log', base=2)
plt.yscale('log', base=10)
plt.title("NCCL and MPI AllGather Communication on Perlmutter\n", fontsize=16)
plt.ylabel('Time (s)')
plt.xlabel('Message Sizes (MB)')
plt.yticks([0.001, 0.01, 0.1, 1], ['0.001', '0.01', '0.1', '1'])
plt.legend(loc='upper left', ncol=2, bbox_to_anchor=(1, 1))
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
set_aspect_ratio(3/5, logx=2, logy=10)

plt.savefig("comm_nccl_mpi_allgather.pdf", bbox_inches='tight')

Matplotlib is building the font cache; this may take a moment.
findfont: Font family ['gill-sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['gill-sans'] not found. Falling back to DejaVu Sans.


0.0 0.00023631651194061168


In [4]:
# MPI-NCCL-ALL-REDUCE

setup_global()
colors_dict = get_colors_dict()
linestyles = get_linestyles_dict()
markers = get_markers()

setup_local()

gpus = [8, 16, 32, 64, 128]
x1 = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]

mpi_times128 = [0.260096, 0.535750, 1.089220, 3.236966, 6.499632, 12.975189]
mpi_times16 = [0.142862, 0.282599, 0.635635, 1.893851, 3.800098, 7.591759]
mpi_times32 = [0.050115, 0.093747, 0.182627, 0.363477, 0.777837, 2.348574, 4.726795, 9.478696]
mpi_times64 = [0.120696, 0.238777, 0.470335, 0.963299, 2.857795, 5.742566, 11.495248]
mpi_times8 = [0.056844, 0.108090, 0.215626, 0.502310, 1.519484, 3.075941, 6.121168]
nccl_times128 = [0.002305, 0.003309, 0.005263, 0.008851, 0.017150, 0.037149, 0.075655]
nccl_times16 = [0.000969, 0.001819, 0.003596, 0.006813, 0.013459, 0.026683, 0.052290]
nccl_times32 = [0.001510, 0.001222, 0.001317, 0.002024, 0.003762, 0.007554, 0.014173, 0.027756, 0.054544]
nccl_times64 = [0.001551, 0.001949, 0.002918, 0.004132, 0.007447, 0.014747, 0.028172, 0.055372]
nccl_times8 = [0.000635, 0.000887, 0.001639, 0.003232, 0.006303, 0.011998, 0.024143, 0.047652]


i=0
plt.plot(x1[1:],nccl_times8, label="8 GPUs NCCL", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[1:-1],mpi_times8, label="8 GPUs MPI", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["dashed"])

i=1
plt.plot(x1[2:], nccl_times16, label="16 GPUs NCCL", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[2:-1], mpi_times16, label="16 GPUs MPI", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["dashed"])

i=2
plt.plot(x1, nccl_times32, label="32 GPUs NCCL", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[:-1], mpi_times32, label="32 GPUs MPI", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["dashed"])

i=3 
plt.plot(x1[1:], nccl_times64, label="64 GPUs NCCL", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x1[1:-1], mpi_times64, label="64 GPUs MPI", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["dashed"])

i=4
plt.plot(x1[2:], nccl_times128, label="128 GPUs NCCL", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x1[2:-1], mpi_times128, label="128 GPUs MPI", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["dashed"])

plt.xscale('log', base=2)
plt.yscale('log', base=10)
plt.title("NCCL and MPI AllReduce Communication on Perlmutter\n", fontsize=16)
plt.ylabel('Time (s)')
plt.xlabel('Message Sizes (MB)')
plt.yticks([0.001, 0.01, 0.1, 1], ['0.001', '0.01', '0.1', '1'])
plt.legend(loc='upper left', ncol=2, bbox_to_anchor=(1, 1))
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
set_aspect_ratio(3/5, logx=2, logy=10)

plt.savefig("comm_nccl_mpi_allreduce.pdf", bbox_inches='tight')

1.328630690979781 0.0003865954398455726


In [5]:
# MPI-NCCL-REDUCE-SCATTER

setup_global()
colors_dict = get_colors_dict()
linestyles = get_linestyles_dict()
markers = get_markers()

setup_local()

gpus = [8, 16, 32, 64, 128]
x1 = [8, 16, 32, 64, 128, 256, 512, 1024, 2048]

mpi_times128 = [0.410163, 0.429161, 0.544002, 0.679339, 0.981913, 1.583797, 3.678590]
mpi_times16 = [0.056117, 0.092396, 0.169070, 0.331578, 0.641127, 1.270086, 3.735213]
mpi_times32 = [0.053765, 0.064537, 0.084740, 0.133787, 0.220573, 0.377243, 0.683938, 1.321649, 3.716915]
mpi_times64 = [0.157345, 0.205494, 0.216133, 0.316748, 0.476547, 0.776507, 1.387122, 3.688627]
mpi_times8 = [0.024237, 0.043589, 0.083173, 0.153300, 0.300631, 0.598284, 1.190578, 3.832743]
nccl_times128 = [0.002055, 0.002314, 0.003003, 0.004164, 0.007515, 0.014791, 0.027948]
nccl_times16 = [0.000552, 0.000933, 0.001772, 0.003462, 0.007059, 0.013749, 0.026539]
nccl_times32 = [0.000586, 0.000629, 0.000712, 0.001141, 0.002012, 0.003715, 0.007022, 0.014078, 0.027699]
nccl_times64 = [0.001059, 0.001147, 0.001410, 0.002090, 0.004116, 0.007125, 0.014305, 0.028156]
nccl_times8 = [0.000363, 0.000450, 0.000876, 0.001650, 0.003169, 0.006491, 0.012103, 0.024166]


i=0
plt.plot(x1[1:],nccl_times8, label="8 GPUs NCCL", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[1:],mpi_times8, label="8 GPUs MPI", color=colors_dict["green"], marker=markers[i], 
         markeredgecolor=colors_dict["green"], clip_on=True, linestyle=linestyles["dashed"])

i=1
plt.plot(x1[2:], nccl_times16, label="16 GPUs NCCL", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1[2:], mpi_times16, label="16 GPUs MPI", color=colors_dict["orange"], marker=markers[i], 
         markeredgecolor=colors_dict["orange"], clip_on=True, linestyle=linestyles["dashed"])

i=2
plt.plot(x1, nccl_times32, label="32 GPUs NCCL", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["solid"])
plt.plot(x1, mpi_times32, label="32 GPUs MPI", color=colors_dict["purple"], marker=markers[i], 
         markeredgecolor=colors_dict["purple"], clip_on=True, linestyle=linestyles["dashed"])

i=3
plt.plot(x1[1:], nccl_times64, label="64 GPUs NCCL", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x1[1:], mpi_times64, label="64 GPUs MPI", color=colors_dict["blue"], marker=markers[i], 
         markeredgecolor=colors_dict["blue"], clip_on=True, linestyle=linestyles["dashed"])

i=4
plt.plot(x1[2:], nccl_times128, label="128 GPUs NCCL", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["solid"]) 
plt.plot(x1[2:], mpi_times128, label="128 GPUs MPI", color=colors_dict["black"], marker=markers[i], 
         markeredgecolor=colors_dict["black"], clip_on=True, linestyle=linestyles["dashed"])

plt.xscale('log', base=2)
plt.yscale('log', base=10)
plt.title("NCCL and MPI Reduce-Scatter Communication on Perlmutter\n", fontsize=16)
plt.ylabel('Time (s)')
plt.xlabel('Message Sizes (MB)')
plt.yticks([0.001, 0.01, 0.1, 1], ['0.001', '0.01', '0.1', '1'])
plt.legend(loc='upper left', ncol=2, bbox_to_anchor=(1, 1))
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
set_aspect_ratio(3/5, logx=2, logy=10)

plt.savefig("comm_nccl_mpi_reduce_scatter.pdf", bbox_inches='tight')

0.7846898528382049 0.00022841597338270567
